# Mocks Targets to Mock Properties

The goal of this notebook is to demonstrate how to use the *target* and *truth* catalogs generated by `select_mock_targets`   
to pull out additional galaxy properties from the parent mock catalogs (which varies with target class).

John Moustakas  
Siena College  
2019 November 3

In [1]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

In [4]:
import os
datadir = '{DESI_ROOT}/datachallenge/test-mockid'.format(**os.environ)

### Preliminaries

First, we need to build a simple dataset of `select_mock_targets` outputs by carrying out the following steps:
1. In an empty directory (here, **datadir**) grab the nominal [select-mock-targets.yaml](https://github.com/desihub/desitarget/blob/0.33.2/py/desitarget/mock/data/select-mock-targets.yaml) file.  In this file, the nominal mock catalogs are:

| Target | Mock Catalog                                                    |
|------------|-------------------------------------------------------------|
| BGS        | `{DESI_ROOT}/mocks/bgs/MXXL/full_sky/v0.0.4/BGS_r20.6.hdf5` |
| ELG        | `{DESI_ROOT}/mocks/DarkSky/v1.0.1/elg_0_inpt.fits`          |
| LRG        | `{DESI_ROOT}/mocks/DarkSky/v1.0.1/lrg_0_inpt.fits`          |
| QSO        | `{DESI_ROOT}/mocks/DarkSky/v1.0.1/qso_0_inpt.fits`          |
| LYA        | `{DESI_ROOT}/mocks/lya_forest/london/v4.2.0/master.fits`    |
| MWS_MAIN   | `{DESI_ROOT}/mocks/mws/galaxia/alpha/v0.0.6/healpix`        |
| MWS_NEARBY | `{DESI_ROOT}/mocks/mws/100pc/v0.0.4/mock_100pc.fits`        |
| WD         | `{DESI_ROOT}/mocks/mws/wd/v1.0.0/mock_wd.fits`              |
| SKY        | `{DESI_ROOT}/mocks/uniformsky/0.2/uniformsky-2048-0.2.fits` |

2. Next, launch the code (here, with 4 cores and a couple randomly selected `nside=128` healpixels and with no spectra, for speed):

```
select_mock_targets -c select-mock-targets.yaml --nproc 4 --verbose --seed 1 \  
  --healpixels 104120 104121 --nside 128 --no-spectra --overwrite
```

Note that this simulation will include contaminants, which we need to handle below.

3. Finally, join the individual `targets` and `truth` catalogs in each healpixel (separately for *bright* and *dark* targets) into the top-level directory with:

```
join_mock_targets --mockdir . --overwrite
```

If each one of these steps worked then your top-level directory should contain the following files:

```
% ls -l *
drwxrwxr-x  4 ioannis  staff      128 Oct 21 14:21 1041/
-rw-rw-r--  1 ioannis  staff   630720 Oct 22 12:33 mtl-bright.fits
-rw-rw-r--  1 ioannis  staff  1008000 Oct 22 12:33 mtl-dark.fits
-rw-r--r--@ 1 ioannis  staff     2488 Oct 21 14:16 select-mock-targets.yaml
-rw-rw-r--  1 ioannis  staff   302400 Oct 22 12:33 sky.fits
-rw-rw-r--  1 ioannis  staff   633600 Oct 22 12:33 targets-bright.fits
-rw-rw-r--  1 ioannis  staff   999360 Oct 22 12:33 targets-dark.fits
-rw-rw-r--  1 ioannis  staff   218880 Oct 22 12:33 truth-bright.fits
-rw-rw-r--  1 ioannis  staff   437760 Oct 22 12:33 truth-dark.fits
```

### Basic imports

In [7]:
import os
import numpy as np
import fitsio

### Read the parent targets and truth catalogs.

You can see that some targets (e.g., BGS, STAR, WD) appear as both bright- *and* dark-time targets, while others (e.g., ELG, LRG, QSO) appear only as dark-time targets.

In [12]:
for obscon in ('dark', 'bright'):
    for cattype in ('targets', 'truth'):
        catfile = os.path.join(datadir, '{}-{}.fits'.format(cattype, obscon))
        print(fitsio.FITS(catfile))


  file: /Users/ioannis/work/desi/datachallenge/test-mockid/targets-dark.fits
  mode: READONLY
  extnum hdutype         hduname[v]
  0      IMAGE_HDU       
  1      BINARY_TBL      TARGETS

  file: /Users/ioannis/work/desi/datachallenge/test-mockid/truth-dark.fits
  mode: READONLY
  extnum hdutype         hduname[v]
  0      IMAGE_HDU       
  1      BINARY_TBL      TRUTH
  2      BINARY_TBL      TRUTH_BGS
  3      BINARY_TBL      TRUTH_ELG
  4      BINARY_TBL      TRUTH_LRG
  5      BINARY_TBL      TRUTH_QSO
  6      BINARY_TBL      TRUTH_STAR
  7      BINARY_TBL      TRUTH_WD

  file: /Users/ioannis/work/desi/datachallenge/test-mockid/targets-bright.fits
  mode: READONLY
  extnum hdutype         hduname[v]
  0      IMAGE_HDU       
  1      BINARY_TBL      TARGETS

  file: /Users/ioannis/work/desi/datachallenge/test-mockid/truth-bright.fits
  mode: READONLY
  extnum hdutype         hduname[v]
  0      IMAGE_HDU       
  1      BINARY_TBL      TRUTH
  2      BINARY_TBL      TRUTH_BGS

In [18]:
targ_dark = fitsio.read(os.path.join(datadir, 'targets-dark.fits'), ext=1)
true_dark = fitsio.read(os.path.join(datadir, 'truth-dark.fits'), ext=1)
targ_bright = fitsio.read(os.path.join(datadir, 'targets-bright.fits'), ext=1)
true_bright = fitsio.read(os.path.join(datadir, 'truth-bright.fits'), ext=1)

### Mapping mock targets to the parent mock catalog. 

Now let's investigate the targets for each target class. The basic idea is that you can use **TARGETID** to cross-reference targets across the various data tables, and that you can use **MOCKID** to find the position (i.e., row) of the target in the parent mock catalog.

In [22]:
class2mockfile = dict(
    BGS='{DESI_ROOT}/mocks/bgs/MXXL/full_sky/v0.0.4/BGS_r20.6.hdf5',
    ELG='{DESI_ROOT}/mocks/DarkSky/v1.0.1/elg_0_inpt.fits',
    LRG='{DESI_ROOT}/mocks/DarkSky/v1.0.1/lrg_0_inpt.fits',
    QSO='{DESI_ROOT}/mocks/DarkSky/v1.0.1/qso_0_inpt.fits',
    LYA='{DESI_ROOT}/mocks/lya_forest/london/v4.2.0/master.fits',
    MWS_MAIN='{DESI_ROOT}/mocks/mws/galaxia/alpha/v0.0.6/healpix',
    MWS_NEARBY='{DESI_ROOT}/mocks/mws/100pc/v0.0.4/mock_100pc.fits',
    WD='{DESI_ROOT}/mocks/mws/wd/v1.0.0/mock_wd.fits',
    SKY='{DESI_ROOT}/mocks/uniformsky/0.2/uniformsky-2048-0.2.fits'
)

### BGS

The BGS mock is one single mock catalog, so we need to read the whole thing.

In [32]:
def read_bgs():
    import h5py
    mockfile = class2mockfile['BGS'].format(**os.environ)
    print('Reading {}'.format(mockfile))
    with h5py.File(mockfile, mode='r') as f:
        ra  = f['Data/ra'][:].astype('f8') % 360.0 # enforce 0 < ra < 360
        dec = f['Data/dec'][:].astype('f8')
        zobs = f['Data/z_obs'][:].astype('f4')    
    return ra, dec, zobs

In [33]:
ra, dec, zobs = read_bgs()

Reading /Users/ioannis/work/desi/mocks/bgs/MXXL/full_sky/v0.0.4/BGS_r20.6.hdf5


Next, read the *BGS* truth catalog, clean out contaminants (using the *TEMPLATETYPE* column), and then pull out the row of each target in the parent mock catalog using the *MOCKID* column.

In [39]:
true_bgs = fitsio.read(os.path.join(datadir, 'truth-bright.fits'), extname='TRUTH_BGS')
print(len(true_bgs), true_bgs.dtype)

1266 [('TARGETID', '>i8'), ('MOCKID', '>i8'), ('TRUEZ', '>f4'), ('TRUESPECTYPE', '<U10'), ('TEMPLATETYPE', '<U10'), ('TEMPLATESUBTYPE', '<U10'), ('TEMPLATEID', '>i4'), ('SEED', '>i8'), ('MAG', '>f4'), ('MAGFILTER', '<U15'), ('FLUX_G', '>f4'), ('FLUX_R', '>f4'), ('FLUX_Z', '>f4'), ('FLUX_W1', '>f4'), ('FLUX_W2', '>f4'), ('FLUX_W3', '>f4'), ('FLUX_W4', '>f4')]


In [44]:
true_bgs['TARGETID']

array([288230812862644224, 288230812862644225, 288230812862644226, ...,
       288230812866839777, 288230812866839778, 288230812866839779])

In [43]:
targ_bright['TARGETID']

array([288230812862644224, 288230812862644225, 288230812862644226, ...,
       288230812866839777, 288230812866839778, 288230812866839779])

In [46]:
jj = fitsio.read(os.path.join(datadir, 'truth-bright.fits'), extname='TRUTH_STAR')
len(jj['TEMPLATEID'])

1266

In [ ]:

true_bgs = true_bgs[true_bgs['TEMPLATETYPE'] == 'BGS']

In [37]:
indxinmock = true_bgs['MOCKID']
assert(np.all(zobs[indxinmock] == true_bgs['TRUEZ']))
assert(np.all(zobs[indxinmock] == true_bgs['TRUEZ']))
assert(np.all(zobs[indxinmock] == true_bgs['TRUEZ']))


True